# Assignment_3_CS350

<ins>Objective</ins>: An analysis of the fraud detection database, for the possibility of second-party fraud.

<ins>Create</ins>: Jupyter notebook for the analysis, in Python

<ins>Steps</ins>: For list of steps required to complete this assignment please visit Class 20 slides, there are 14 steps.

## Step 11

In [ ]:
from neo4j import GraphDatabase

# Neo4j connection details
uri = "bolt://localhost:7689"  # Replace with your Neo4j URI
username = "neo4j"  # Replace with your Neo4j username
password = "password"  # Replace with your Neo4j password
driver = GraphDatabase.driver(uri, auth=(username, password))

# Function to write the relationships
def write_relationships(tx, graph_name):
    query = """
    CALL gds.graph.writeRelationship(
        $graph_name,
        'SIMILAR_TO',
        'jaccardScore'
    )
    YIELD relationshipsWritten
    RETURN relationshipsWritten
    """
    result = tx.run(query, graph_name=graph_name)
    return result.single()

# Run the node similarity algorithm in mutate mode
with driver.session() as session:
    result = session.execute_write(write_relationships, graph_name='similarity')
    print(f"Relationships Created: {result['relationshipsWritten']}")

## Step 12

In [ ]:
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

# Neo4j connection details
uri = "bolt://localhost:7689"  # Replace with your Neo4j URI
username = "neo4j"  # Replace with your username
password = "password"  # Replace with your password

# Create a Neo4j driver and GDS instance
driver = GraphDatabase.driver(uri, auth=(username, password))
gds = GraphDataScience(driver)

# Create a function to run the gds.degree.write procedure
def compute_degree_and_write(tx, graph_name):
    query = """
    CALL gds.degree.write(
        $graph_name,
        {
            nodeLabels: ['Client'],
            relationshipTypes: ['SIMILAR_TO'],
            relationshipWeightProperty: 'jaccardScore',
            writeProperty: 'secondPartyFraudScore'
        }
    )
    """
    tx.run(query, graph_name='similarity')

# Run the degree calculation and writing procedure in WRITE mode
with driver.session() as session:
    session.execute_write(compute_degree_and_write, 'similarity')
    print("Second party fraud property successfully written.")

## Step 13

In [ ]:
from neo4j import GraphDatabase

# Neo4j connection details
uri = "bolt://localhost:7689"  # Replace with your Neo4j URI
username = "neo4j"  # Replace with your Neo4j username
password = "password"  # Replace with your Neo4j password
driver = GraphDatabase.driver(uri, auth=(username, password))

# Function to label potential fraudsters based on degree of centrality
def label_potential_fraudster(tx, graph_name):
    query = """
    MATCH (c:Client)
    WHERE c.secondPartyFraudScore IS NOT NULL
    WITH percentileCont(c.secondPartyFraudScore, 0.95) AS threshold
        MATCH (c:Client)
        WHERE c.secondPartyFraudScore > threshold
        SET c.SecondPartyFraudster = true
    """
    tx.run(query, graph_name=graph_name)

# Execute the labeling within a write transaction
with driver.session() as session:
    session.execute_write(label_potential_fraudster, graph_name='similarity')
    print("Successfully labelled potential fraudsters with a high degree of centrality.")

## Step 14

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

# Neo4j connection details
uri = "bolt://localhost:7689"  # Replace with your Memgraph URI
username = "neo4j"  # Replace with your Neo4j username
password = "password"  # Replace with your Neo4j password
driver = GraphDatabase.driver(uri, auth=(username, password))

# Function to fetch fraudster clients and return as a Pandas DataFrame
def fetch_fraudster_clients(tx):
    query = """
    MATCH (c:Client)
    WHERE c.SecondPartyFraudster = true
    RETURN c.name AS clientName, ID(c) AS clientId
    """
    result = tx.run(query)
    
    # Collect results into a list of dictionaries to make it easier to import into a Pandas DataFrame
    clients = [{"clientName": record["clientName"], "clientId": record["clientId"]} for record in result]
    
    # Convert to Pandas DataFrame
    df = pd.DataFrame(clients)
    return df

# Execute the query within a read transaction
with driver.session() as session:
    fraudster_df = session.execute_read(fetch_fraudster_clients)
    
    # Display the results in the notebook using Pandas
    if not fraudster_df.empty:
        print("List of potential fraudsters:")
        display(fraudster_df)  # display() is used in Jupyter to render the DataFrame
    else:
        print("No potential fraudsters found.")
